In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import plotly.graph_objs as go
import pandas as pd
import pyfolio
import warnings

import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")


def resample_to_constant_volume(df, volume_per_candle, convertTime= False):
    # sort by time
    #df = df.sort_values(by='time')
    # create cumulative volume column
    df['cumulative_volume'] = df['volume'].cumsum()
    
    #print(len(df))
    # initialize the new dataframe to hold resampled data
    resampled_df = pd.DataFrame(columns=['open', 'high', 'low', 'close','volume',"date"])
    
    start_index = 0
    end_index = 0
    cumulative_volume = 0
    for i in range(len(df)):
        cumulative_volume += df.iloc[i]['volume']
        end_index = i
        if cumulative_volume >= volume_per_candle : # make function dynamic in next round
            candle_df = df.iloc[start_index:end_index+1]
            open_price = candle_df.iloc[0]['open']
            high_price = candle_df['high'].max()
            low_price = candle_df['low'].min()
            close_price = candle_df.iloc[-1]['close']
            date_end = df.iloc[start_index]["date"]
            #x =  pd.to_datetime(datetime.strptime(mask, '%Y-%m-%d %H:%M:%S'))
            ## similarly for date_end,, this is to quickly find what 5 minute candle it belong to
            #date_end = x.strftime(f"%Y-%m-%d %H:{5*int(x.minute/5)}:%S")
            if convertTime:
                mask = df.iloc[end_index]["date"].split("+")[0]
                x =  pd.to_datetime(datetime.strptime(mask, '%Y-%m-%d %H:%M:%S'))
                ## similarly for date_end,, this is to quickly find what 5 minute candle it belong to
                temp = ("0"+str(5*int(x.minute/5)))
                date_end = x.strftime(f"%Y-%m-%d %H:{temp[-2:]}:%S")
            """resampled_df = resampled_df.append({'open': open_price, 'high': high_price, 'low': low_price, 
                                                'close': close_price, 'volume': cumulative_volume,
                                                 "date_end": date_end}, 
                                                 ignore_index=True)"""
            for i in range(int(1)):
                resampled_df = resampled_df.append({'open': open_price, 'high': high_price, 'low': low_price, 
                                                    'close': close_price, 'volume': cumulative_volume,
                                                     "date": date_end}, 
                                                     ignore_index=True)
            cumulative_volume = 0
            start_index = end_index + 1
    
    if start_index < len(df):
        candle_df = df.iloc[start_index:len(df)]
        open_price = candle_df.iloc[0]['open']
        high_price = candle_df['high'].max()
        low_price = candle_df['low'].min()
        close_price = candle_df.iloc[-1]['close']
        volume = sum(candle_df["volume"])
        date_end = df.iloc[start_index]["date"]
        #x =  pd.to_datetime(datetime.strptime(mask, '%Y-%m-%d %H:%M:%S'))
        ## similarly for date_end,, this is to quickly find what 5 minute candle it belong to
        #date_end = x.strftime(f"%Y-%m-%d %H:{5*int(x.minute/5)}:%S")
        if convertTime:
            mask = df.iloc[end_index]["date"].split("+")[0]
            x =  pd.to_datetime(datetime.strptime(mask, '%Y-%m-%d %H:%M:%S'))
            ## similarly for date_end,, this is to quickly find what 5 minute candle it belong to
            temp = ("0"+str(5*int(x.minute/5)))
            date_end = x.strftime(f"%Y-%m-%d %H:{temp[-2:]}:%S")
        resampled_df = resampled_df.append({'open': open_price, 'high': high_price, 'low': low_price, 'close': close_price, 'volume': volume,"date": str(date_end)}, ignore_index=True)

    return resampled_df
from datetime import timedelta
def is_body_size_greater_than_nx_average(dataframe, index,atr_multiple,look_back):
    body_sizes = (dataframe['close'] - dataframe['open']).loc[index - look_back:index].abs()
    #print(body_sizes,"efb")
    average_body_size = body_sizes.mean()
    #print(average_body_size)
    current_body_size = abs(dataframe['close'].loc[index] - dataframe['open'].loc[index])
    return current_body_size > atr_multiple * average_body_size


def imbalance_from_volume(df, dateTime_index_to_check, context = {}):
    
    ind = df[df["date"] == dateTime_index_to_check].index[0]
   
    
#     x =  pd.to_datetime(datetime.strptime(dateTime_index_to_check, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=1))
#     endTime = x.strftime(f"%Y-%m-%d %H:%M:%S")

#     end_index = (df[df["date"] == endTime].index)
#     if len(end_index):
#         end_index=end_index[0] ## corner case 15:30
#     else:
#         return False
    
    volume_per_candle = df["volume"].rolling(window=200).mean().rolling(window=30).quantile(0.7).loc[ind]
    if context:
        vol_resampled_data1 = resample_to_constant_volume(context["resampled_candles"], volume_per_candle ,convertTime = True)
        context["resampled_candles"] = vol_resampled_data1
   
    
#     index_in_vol = vol_resampled_data1.index[-1]#vol_resampled_data1[vol_resampled_data1["date_end"] == dateTime_index_to_check].index[0]
    #print(index_in_vol,"jk")
    #print((vol_resampled_data1['volume'][index_in_vol]),volume_per_candle)
    if(((df['volume'][ind])/volume_per_candle) > 1.3): 
        return True
    else :
        return False
def is_body_size_greater_than_3x_average(dataframe, index,atr_multiple,look_back):   #price-time 
    body_sizes = (dataframe['close'] - dataframe['open']).loc[index - look_back:index].abs()
    #print(body_sizes)
    average_body_size = body_sizes.mean()
    #print(average_body_size,"Wrv__________")
    current_body_size = abs(dataframe['close'].loc[index] - dataframe['open'].loc[index])
    #print(current_body_size,"current")
    return current_body_size > atr_multiple * average_body_size

def imbalance_zone(df,dateTime_index_to_check,look_back=30):   #return imbalance zone ( price levels)
    imbalance_candle = df[df["date"] == dateTime_index_to_check].index[0]
    verify_candle  = imbalance_candle+1
    current_candle = imbalance_candle+2

    
    if is_body_size_greater_than_3x_average(df,imbalance_candle,1.2,look_back):
        upper = min(df["low"].loc[imbalance_candle-look_back:imbalance_candle-1])
        lower = df["high"].loc[verify_candle]
        #print(upper,lower)

        if (upper-lower) > 0 :
            return [ lower , upper ]  ## test in future the difference of lines more than some % of body

        else :
            return [0 , 0]
    else:
        return [0 , 0]
def filter_candles_at_index(df, index, body_threshold = 10, untouched_threshold = 20):
    # Calculate the body of the specified candle
    #index = df[df["date"] == dateTime_index_to_check].index[0]
#     print(index)
    df['body'] = df['close'] - df['open']
    body_threshold = (body_threshold+1)*df["body"].mean()
#     print(df["body"])
    body = df.loc[index]['body']
#     print(df)
    # Select only the specified candle if the body is high
    if abs(body) >= body_threshold:
        # Find the percentage of the price range covered by the body that has not been touched by the previous 10 candles
        df['body_min'] = df[['open', 'close']].min(axis=1)
        df['body_max'] = df[['open', 'close']].max(axis=1)
        df['prev_min'] = df['body_min'].shift(1).rolling(window=10).min()
        df['prev_max'] = df['body_max'].shift(1).rolling(window=10).max()
        df['price_range_untouched_min'] = df['body_min'].where(df['body_min'] > df['prev_max'], 0)
        df['price_range_untouched_max'] = df['body_max'].where(df['body_max'] < df['prev_min'], 0)
        df['price_range_untouched'] = df[['price_range_untouched_min', 'price_range_untouched_max']].max(axis=1) - df[['price_range_untouched_min', 'price_range_untouched_max']].min(axis=1)
        df['untouched_percentage'] = df['price_range_untouched'].abs() / df['body'].abs()
        # Select only the specified candle if the percentage is large
        if df.at[index, 'untouched_percentage'] >= untouched_threshold:
            return True
        else:
            return False
    else:
        return False
    
def volCheck(df,dateTime_index_to_check):
    ind = df[df["date"] == dateTime_index_to_check].index[0]
    x =  pd.to_datetime(datetime.strptime(dateTime_index_to_check, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=5))
    #temp = ("0"+str(5*int((x.minute+5)/5)))
    endTime = x.strftime(f"%Y-%m-%d %H:%M:%S")
    # ind2 = df[df["date"] == endTime].index[0]
    # df = df[:ind+1]
    #print(endTime)
    end_index = (df[df["date"] == endTime].index)
    if len(end_index):
        end_index=end_index[0] ## corner case 15:30
    else:
        return False
    
    #print(data["volume"].rolling(window=80000).mean()[ind])
    #vol_resampled_data = resample_to_constant_volume(df[ind-2000:end_index],df["volume"].rolling(window=80000).mean()[ind]*1.7)
    
    volume_per_candle = df["volume"].rolling(window=8000).quantile(0.95).loc[ind]*2.5
    vol_resampled_data1 = resample_to_constant_volume(df.loc[ind-2000:end_index], volume_per_candle ,convertTime = True)
    index = vol_resampled_data1.index[-1]
    return filter_candles_at_index(vol_resampled_data1,index)

    

from collections import defaultdict

def to_intervals(data,n):   # to convert it n minute candle 
    # Read in the data from the text file
    data
    data = data.reset_index()
    data = np.array(data)
    """data = []
    name = "data_2017.txt" #input()
    with open(name, 'r') as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            # Ignore the first column (stock name)
            data.append(row[1:])"""

    # Convert the data to n-minute intervals
    interval_data =  defaultdict(list)
    for row in data:
        date = row[1].split()[0]
        time = row[1].split()[1].split("+")[0][:-3]
        if time == "09:08:00":
            continue

        open_price = float(row[2])
        high_price = float(row[3])
        low_price = float(row[4])
        close_price = float(row[5])
        volume = float(row[6])
        # Extract the interval from the time and use it as the key in the dictionary
        interval = int(time[-2:]) // n
        interval = f"{time[:2]}:{interval * n:02d}"
        interval = str(date) +" " + interval + ":00"
        interval_data[interval].append((date, time, open_price, high_price, low_price, close_price, volume))

    # Calculate the final open, high, low, and close prices for each interval
    final_prices = {}
    for interval, prices in interval_data.items():
        # Unpack the tuples into separate lists for each price type
        dates, times, opens, highs, lows, closes, volume = zip(*prices)
        # The final open price is the same as the first open price in the interval
        final_open = opens[0]

        # The final high price is the highest of all the high prices in the interval
        final_high = max(highs)
        # The final low price is the lowest of all the low prices in the interval
        final_low = min(lows)
        # The final close price is the same as the last close price in the interval
        final_close = closes[-1]
        # The final volume will be the sum of all volumes of the candels
        final_volume = sum(volume)
        # Store the final prices in a tuple in the dictionary
        final_prices[interval] = (dates[0], times[0], final_open, final_high, final_low, final_close, final_volume)
    
    ws = []
    # Write the data rows
    for interval, final in final_prices.items():
        date, time, open_price, high_price, low_price, close_price, volume = final
        ws.append([date, time, interval, open_price, high_price, low_price, close_price, volume])
    return pd.DataFrame(ws,columns = ["Date", "Time", "date", "open", "high", "low", "close","volume"])
    

def handle_data(context,data):
    ### check on index n-1::
    ## Fetch n-1 candles volume candles:::
    #data = data.reset_index()
    
    ## exit prev conditions
    if context["position"] == True:
        if data["close"].iloc[-1]>=sum(context["priceLevels"])/2:
            prof = data["close"].iloc[-1] - context["buy"]
            context["profit"] += prof
            if prof<0:
                context["losses"] += 1   
            context["profitList"].append(context["profit"])
            context["position"]  = 0
            context["trades"] += 1
        elif data["close"].iloc[-1]<=context["stoploss"]:
            context["losses"] += 1
            context["position"] = 0
            context["profit"] += data["close"].iloc[-1] - context["buy"]
            context["profitList"].append(context["profit"])
            context["trades"] += 1
        if context["position"]==0:
            print("squaredOff",data["date"].iloc[-1])
        return
    #data_5min = to_intervals(data[['date',"open",'high',"low","close","volume"]].iloc[-5000:],5)
    
    candleToCheck = data["date"].iloc[-5]

    if not imbalance_from_volume(data,candleToCheck,context):
        return
#     print("imbalance Found in volume!!")
#     print(candleToCheck,"__________")
#     print(data_5min)
    p1,p2 = imbalance_zone(data, candleToCheck , 30) ##zone funciton....
#     print(p1,p2)
    if p1==p2:
        return
    
    if max(data["high"].iloc[-4:])>(p1):#-data["low"].iloc[-4:]
        return
    if data["close"].iloc[-1]>p1:
        return
    if (data["close"].iloc[-1]-data["open"].iloc[-1])<0:#p1*0.99:
        return
    ## apply price conditions 
    ## between previous 30 5 minute candles low, current candle high -- range of body of imbalance greater than threshold
    ## 5 min data
    
    context["priceLevels"] = [p1,p2]
   
    # strategy_trade(context,data_5min,candleToCheck) #TODO send volume data
    context["imbalanceCandles"].append(candleToCheck)
    next_candle_strategy_trade(context , data)

def buy(context,price):
    context["position"] = True
    context["buy"] = price
    context['stoploss'] = price - 0.002*price   # to be change 

def next_candle_strategy_trade( context , data):
    print("buy order",data["date"].iloc[-1])
    buy(context,data["close"].iloc[-1])


def strategy_trade(context,df,candleToCheck):
    data = df.iloc[-50:]
    data["mid"] = (data["high"]+data["low"])/2
    data["ao"] = data["mid"].rolling(window = 5).mean() - data["mid"].rolling(window = 34).mean()
    signals = data[(data["ao"].shift(4)< data["ao"].shift(3)) & (data["ao"].shift(3)< data["ao"].shift(2)) & (data["ao"].shift(2) > data["ao"])]
    if len(signal):
        ind = signal.index[-1]
        datetime_ = signal["datetime"].loc[ind]  #latest or not ==== send data only after signal
        # datetime.datetime.strptime(datetime_,"")>candleToCheck:

        #TODO 

        buy(context,data["open"].iloc[-1])
        
    #data[data["ao"].shift(3)< data["ao"].shift(2) and data["ao"].shift(2)< data["ao"].shift(1) and data["ao"].shift(1) > data["ao"]]

def plot(df3):
    
    # Create the candlestick chart
    candlestick = go.Candlestick(x=df3.index, open=df3['open'], close=df3['close'], high=df3['high'], low=df3['low'])

    # Add a line plot with secondary x-axis
    #line = go.Scatter(x=df3['date'], y=df3['close'], name='date', xaxis='x2')
    #line = go.Bar(x=df3.index, y=df3['volume'], name='Volume')

    # Create the figure with both the candlestick and line plots
    fig = go.Figure(data=[candlestick])

    # Update the layout to add a secondary x-axis
#     fig.update_layout(xaxis2=dict(title='volume', overlaying='x', side='top'),yaxis2 = dict(title='volume', overlaying='y', side='top'))
    fig.update_layout(xaxis_rangeslider_visible=False)

    # Show the chart
    fig.show()
#     return
def plot_candlestick_with_volume(df3):
    candlestick = go.Candlestick(x=df3.index, open=df3['open'], close=df3['close'], high=df3['high'], low=df3['low'])

    # Add a line plot with secondary x-axis
    #line = go.Scatter(x=df3['date'], y=df3['close'], name='date', xaxis='x2')
    line = go.Bar(x=df3.index, y=df3['volume'], name='Volume')

    # Create the figure with both the candlestick and line plots
    fig = go.Figure(data=[candlestick,line])

    # Update the layout to add a secondary x-axis
    fig.update_layout(xaxis2=dict(title='volume', overlaying='x', side='top'),yaxis2 = dict(title='volume', overlaying='y', side='top'))
    fig.update_layout(xaxis_rangeslider_visible=False)

    # Show the chart
    fig.show()
    return

def filter_candles_at_index(df, index, body_threshold, untouched_threshold):
    # Calculate the body of the specified candle
    df['body'] = df['close'] - df['open']
    body = df.at[index, 'body']
    # Select only the specified candle if the body is high
    if abs(body) >= body_threshold:
        # Find the percentage of the price range covered by the body that has not been touched by the previous 10 candles
        df['body_min'] = df[['open', 'close']].min(axis=1)
        df['body_max'] = df[['open', 'close']].max(axis=1)
        df['prev_min'] = df['body_min'].shift(1).rolling(window=10).min()
        df['prev_max'] = df['body_max'].shift(1).rolling(window=10).max()
        df['price_range_untouched_min'] = df['body_min'].where(df['body_min'] > df['prev_max'], 0)
        df['price_range_untouched_max'] = df['body_max'].where(df['body_max'] < df['prev_min'], 0)
        df['price_range_untouched'] = df[['price_range_untouched_min', 'price_range_untouched_max']].max(axis=1) - df[['price_range_untouched_min', 'price_range_untouched_max']].min(axis=1)
        df['untouched_percentage'] = df['price_range_untouched'].abs() / df['body'].abs()
        # Select only the specified candle if the percentage is large
        if df.at[index, 'untouched_percentage'] >= untouched_threshold:
            return True
        else:
            return False
    else:
        return False

    
def filter_candles(df, body_threshold=10, untouched_threshold=20):
    
    # Calculate the body of each candle
    df['body'] = df['close'] - df['open']
    body_threshold = (body_threshold+1)*df["body"].mean()
    # Select only those candles where the body is high
    df = df[df['body'].abs() >= body_threshold]
    # Find the percentage of the price range covered by the body that has not been touched by the previous 10 candles
    df['body_min'] = df[['open', 'close']].min(axis=1)
    df['body_max'] = df[['open', 'close']].max(axis=1)
    df['prev_min'] = df['body_min'].shift(1).rolling(window=10).min()
    df['prev_max'] = df['body_max'].shift(1).rolling(window=10).max()
    df['price_range_untouched_min'] = df['body_min'].where(df['body_min'] > df['prev_max'], 0)
    df['price_range_untouched_max'] = df['body_max'].where(df['body_max'] < df['prev_min'], 0)
    df['price_range_untouched'] = df[['price_range_untouched_min', 'price_range_untouched_max']].max(axis=1) - df[['price_range_untouched_min', 'price_range_untouched_max']].min(axis=1)
    df['untouched_percentage'] = df['price_range_untouched'].abs() / df['body'].abs()
    # Select only those candles where the percentage is large
    df = df[df['untouched_percentage'] >= untouched_threshold]
    return df
print("adv")

C:\Anaconda\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


adv


In [2]:
df_1min = pd.read_csv('ACC_minute_data_with_indicators.csv')

In [3]:
df_1min = df_1min[['date' , 'open' , 'high', 'low' , 'close' , 'volume']]
df_1min = df_1min.iloc[249971-80000 :654674]
df_1min.head()

,date,open,high,low,close,volume
169971,2017-01-25 11:10:00+05:30,1364.4,1364.40,1364.00,1364.40,232
169972,2017-01-25 11:11:00+05:30,1364.4,1364.40,1364.40,1364.40,693
169973,2017-01-25 11:12:00+05:30,1364.4,1366.50,1364.40,1366.50,740
169974,2017-01-25 11:13:00+05:30,1366.5,1367.75,1366.45,1367.40,1264
169975,2017-01-25 11:14:00+05:30,1367.4,1367.45,1367.40,1367.45,247


In [4]:
def ss(x):
    return datetime.strptime(x,"%Y-%m-%d %H:%M:%S+05:30").strftime("%Y-%m-%d %H:%M:%S")

df_1min["date"]=list(map(ss,df_1min["date"]))

In [5]:
# for i in dic:
#     print(i)

In [13]:
t = df_1min[df_1min['date'] == '2020-08-04 09:57:00'].index[0]
t

470122

In [18]:

check_df = df_1min.loc[t-30 :t+10]
plot(check_df)


In [21]:
# dic = {'2017-12-27 14:43:00': [1733.55, 1732.0],
#   '2017-12-29 11:59:00': [1762.5, 1759.6],
#   '2018-01-01 12:07:00': [1748.4, 1746.85],
#   '2018-01-01 13:08:00': [1748.0, 1746.3],
#   '2018-01-01 14:55:00': [1742.5, 1741.0],
#   '2018-01-01 15:06:00': [1732.05, 1731.3],
#   '2018-01-05 10:25:00': [1807.0, 1805.7],
#   '2018-01-05 13:55:00': [1803.55, 1802.0],
#   '2018-01-11 13:46:00': [1810.05, 1807.8],
#   '2018-01-12 10:08:00': [1808.0, 1802.75],
#   '2018-01-12 12:33:00': [1802.2, 1800.8],
#   '2018-01-18 13:50:00': [1840.65, 1837.85],
#   '2018-01-18 14:48:00': [1835.8, 1833.95],
#   '2018-01-19 12:26:00': [1821.0, 1818.65],
#   '2018-01-23 13:11:00': [1802.25, 1798.95],
#   '2018-01-23 14:12:00': [1791.5, 1789.95],
#   '2018-01-24 10:00:00': [1771.0, 1770.65],
#   '2018-01-24 10:24:00': [1768.0, 1766.35],
#   '2018-01-24 12:26:00': [1752.35, 1751.95],
#   '2018-01-29 14:06:00': [1747.0, 1743.95],
#   '2018-01-29 14:12:00': [1742.6, 1742.0],
#   '2018-01-31 10:01:00': [1727.15, 1722.45],
#   '2018-01-31 13:56:00': [1705.0, 1702.95],
#   '2018-02-05 11:40:00': [1618.25, 1616.5],
#   '2018-02-07 13:28:00': [1601.0, 1599.0],
#   '2018-02-08 11:33:00': [1590.15, 1588.05],
#   '2018-02-08 14:20:00': [1601.75, 1599.0],
#   '2018-02-09 14:58:00': [1662.0, 1660.45],
#   '2018-02-14 11:07:00': [1641.85, 1639.5],
#   '2018-02-14 11:17:00': [1633.35, 1632.15],
#   '2018-02-14 15:22:00': [1626.85, 1625.9],
#   '2018-02-15 11:03:00': [1645.0, 1644.9],
#   '2018-02-15 12:12:00': [1641.0, 1639.25],
#   '2018-02-15 13:08:00': [1640.0, 1638.45],
#   '2018-02-15 14:39:00': [1624.0, 1622.45],
#   '2018-02-15 14:48:00': [1620.05, 1616.4],
#   '2018-02-19 09:58:00': [1645.4, 1643.5],
#   '2018-02-19 12:30:00': [1637.05, 1635.35],
#   '2018-02-19 13:34:00': [1627.25, 1626.75],
#   '2018-02-20 10:28:00': [1636.1, 1635.9],
#   '2018-02-20 10:43:00': [1630.1, 1629.25],
#   '2018-02-21 13:22:00': [1654.1, 1652.45],
#   '2018-02-21 14:45:00': [1650.0, 1646.95],
#   '2018-02-26 14:18:00': [1663.9, 1661.7],
#   '2018-02-27 10:04:00': [1626.0, 1625.0],
#   '2018-02-28 11:35:00': [1624.6, 1623.0],
#   '2018-02-28 15:07:00': [1625.0, 1624.5],
#   '2018-03-06 09:49:00': [1608.1, 1605.85],
#   '2018-03-06 14:06:00': [1606.05, 1605.05],
#   '2018-03-06 14:11:00': [1603.0, 1602.05],
#   '2018-03-06 14:14:00': [1600.1, 1598.6],
#   '2018-03-07 10:38:00': [1585.1, 1584.5],
#   '2018-03-07 13:39:00': [1580.0, 1579.25],
#   '2018-03-07 14:06:00': [1574.0, 1573.9],
#   '2018-03-08 10:19:00': [1556.35, 1555.4],
#   '2018-03-09 11:12:00': [1536.65, 1535.3],
#   '2018-03-09 13:56:00': [1530.0, 1529.0],
#   '2018-03-19 12:19:00': [1548.4, 1548.0],
#   '2018-03-19 13:48:00': [1548.95, 1547.7],
#   '2018-03-20 11:05:00': [1552.95, 1549.3],
#   '2018-03-20 13:05:00': [1544.0, 1543.35],
#   '2018-03-20 14:12:00': [1535.6, 1535.05],
#   '2018-03-21 12:27:00': [1561.0, 1559.95],
#   '2018-03-21 12:34:00': [1558.05, 1557.0],
#   '2018-03-21 13:49:00': [1555.5, 1554.0],
#   '2018-03-21 13:57:00': [1552.75, 1551.0],
#   '2018-03-27 10:04:00': [1533.5, 1531.65],
#   '2018-03-27 11:29:00': [1528.35, 1527.8],
#   '2018-03-27 14:49:00': [1518.0, 1517.0],
#   '2018-04-04 13:26:00': [1546.6, 1545.0],
#   '2018-04-06 11:09:00': [1545.8, 1545.25],
#   '2018-04-06 12:51:00': [1547.5, 1546.6],
#   '2018-04-09 13:17:00': [1545.0, 1542.75],
#   '2018-04-10 10:48:00': [1541.0, 1539.55],
#   '2018-04-10 13:56:00': [1535.25, 1534.5],
#   '2018-04-12 13:20:00': [1527.1, 1526.55],
#   '2018-04-13 10:28:00': [1530.15, 1529.2],
#   '2018-04-17 11:17:00': [1581.35, 1579.35],
#   '2018-04-17 12:45:00': [1580.2, 1579.9],
#   '2018-04-18 11:39:00': [1586.0, 1584.9],
#   '2018-04-18 13:15:00': [1583.0, 1582.0],
#   '2018-04-18 15:01:00': [1581.0, 1575.7],
#   '2018-04-19 11:02:00': [1602.5, 1601.25],
#   '2018-04-19 14:18:00': [1597.5, 1596.15],
#   '2018-04-19 14:47:00': [1590.0, 1587.6],
#   '2018-04-19 14:57:00': [1577.05, 1573.95],
#   '2018-04-20 11:15:00': [1573.0, 1570.3],
#   '2018-04-20 13:48:00': [1570.5, 1568.95],
#   '2018-04-23 14:06:00': [1570.6, 1568.5],
#   '2018-04-24 11:15:00': [1567.7, 1567.05],
#   '2018-04-24 12:43:00': [1561.0, 1560.0],
#   '2018-04-25 12:49:00': [1565.95, 1565.25],
#   '2018-04-25 14:27:00': [1560.0, 1558.45],
#   '2018-04-25 14:35:00': [1555.95, 1554.45],
#   '2018-04-26 10:02:00': [1561.0, 1560.65],
#   '2018-04-26 11:37:00': [1560.0, 1558.55],
#   '2018-04-26 14:18:00': [1547.05, 1546.4],
#   '2018-04-26 14:24:00': [1544.0, 1543.3],
#   '2018-04-30 12:03:00': [1592.5, 1591.35],
#   '2018-04-30 14:12:00': [1587.0, 1586.45],
#   '2018-04-30 14:38:00': [1585.05, 1583.4],
#   '2018-05-02 11:40:00': [1573.1, 1571.3],
#   '2018-05-02 14:48:00': [1557.0, 1553.35],
#   '2018-05-03 11:09:00': [1548.45, 1544.95],
#   '2018-05-03 13:53:00': [1548.3, 1546.05],
#   '2018-05-03 14:08:00': [1545.6, 1544.95],
#   '2018-05-03 14:26:00': [1540.1, 1536.45],
#   '2018-05-04 14:17:00': [1540.55, 1539.95],
#   '2018-05-04 14:51:00': [1533.05, 1529.85],
#   '2018-05-07 14:03:00': [1521.6, 1519.55],
#   '2018-05-08 13:00:00': [1526.0, 1524.95],
#   '2018-05-09 10:52:00': [1513.85, 1512.5],
#   '2018-05-09 11:45:00': [1504.9, 1504.0],
#   '2018-05-10 11:29:00': [1500.0, 1496.6],
#   '2018-05-10 11:38:00': [1495.0, 1494.15],
#   '2018-05-10 11:47:00': [1490.0, 1487.95],
#   '2018-05-11 12:16:00': [1490.0, 1486.75],
#   '2018-05-11 12:39:00': [1484.15, 1482.5],
#   '2018-05-11 13:04:00': [1477.0, 1474.65],
#   '2018-05-11 13:09:00': [1472.2, 1469.85],
#   '2018-05-14 10:17:00': [1467.5, 1463.95],
#   '2018-05-14 10:58:00': [1455.25, 1453.4],
#   '2018-05-15 09:32:00': [1436.5, 1433.55],
#   '2018-05-15 13:41:00': [1415.05, 1413.65],
#   '2018-05-15 14:19:00': [1410.0, 1408.0],
#   '2018-05-21 11:41:00': [1346.1, 1344.85],
#   '2018-05-21 12:24:00': [1340.1, 1338.0],
#   '2018-05-21 13:07:00': [1334.5, 1333.8],
#   '2018-05-21 13:16:00': [1331.3, 1331.0],
#   '2018-05-22 13:47:00': [1321.0, 1319.3],
#   '2018-05-24 09:52:00': [1301.5, 1299.9],
#   '2018-05-24 09:57:00': [1296.9, 1296.6],
#   '2018-05-25 09:53:00': [1320.0, 1316.8],
#   '2018-05-25 11:01:00': [1315.0, 1312.85],
#   '2018-05-28 09:38:00': [1320.75, 1319.25],
#   '2018-05-28 15:07:00': [1328.65, 1328.45],
#   '2018-05-28 15:10:00': [1325.25, 1323.55],
#   '2018-05-29 12:40:00': [1305.5, 1304.3],
#   '2018-05-30 15:21:00': [1309.25, 1309.0],
#   '2018-06-01 13:42:00': [1330.15, 1329.8],
#   '2018-06-01 15:15:00': [1330.0, 1329.7],
#   '2018-06-04 14:06:00': [1319.05, 1316.0],
#   '2018-06-04 15:03:00': [1308.4, 1307.0],
#   '2018-06-05 09:40:00': [1286.2, 1285.5],
#   '2018-06-05 10:18:00': [1276.05, 1275.95],
#   '2018-06-05 14:53:00': [1283.05, 1281.7],
#   '2018-06-06 14:35:00': [1305.2, 1304.95],
#   '2018-06-07 11:25:00': [1320.1, 1318.95],
#   '2018-06-07 14:09:00': [1320.6, 1319.95],
#   '2018-06-07 15:23:00': [1315.0, 1314.8],
#   '2018-06-08 10:47:00': [1315.25, 1314.0],
#   '2018-06-08 13:07:00': [1312.1, 1312.0],
#   '2018-06-11 13:25:00': [1332.1, 1331.8],
#   '2018-06-11 15:05:00': [1327.6, 1327.2],
#   '2018-06-12 10:42:00': [1315.0, 1314.1],
#   '2018-06-13 11:32:00': [1338.0, 1337.0],
#   '2018-06-13 14:51:00': [1330.15, 1330.0],
#   '2018-06-14 09:55:00': [1322.8, 1320.7],
#   '2018-06-14 10:08:00': [1320.0, 1319.45],
#   '2018-06-14 12:06:00': [1315.0, 1314.55],
#   '2018-06-14 13:23:00': [1311.0, 1309.65],
#   '2018-06-15 13:05:00': [1300.0, 1298.95],
#   '2018-06-18 11:09:00': [1288.85, 1288.5],
#   '2018-06-19 11:15:00': [1285.0, 1282.8],
#   '2018-06-19 14:22:00': [1282.85, 1281.9],
#   '2018-06-21 14:37:00': [1272.0, 1270.3],
#   '2018-06-25 14:30:00': [1319.0, 1317.45],
#   '2018-06-26 13:25:00': [1355.0, 1353.7],
#   '2018-06-27 10:05:00': [1337.0, 1336.85],
#   '2018-06-27 12:57:00': [1323.05, 1321.85],
#   '2018-06-28 12:30:00': [1307.0, 1306.45],
#   '2018-06-28 12:45:00': [1305.0, 1304.5],
#   '2018-06-28 15:02:00': [1302.05, 1293.3],
#   '2018-06-29 11:22:00': [1346.0, 1344.9],
#   '2018-06-29 14:26:00': [1334.0, 1332.8],
#   '2018-07-03 14:54:00': [1333.05, 1332.7],
#   '2018-07-05 12:12:00': [1388.75, 1387.9],
#   '2018-07-05 13:20:00': [1386.0, 1385.75],
#   '2018-07-05 15:02:00': [1390.0, 1386.3],
#   '2018-07-06 10:49:00': [1401.1, 1394.9],
#   '2018-07-06 12:51:00': [1390.6, 1388.95],
#   '2018-07-06 15:03:00': [1378.0, 1376.75],
#   '2018-07-09 14:21:00': [1383.05, 1381.5],
#   '2018-07-09 14:45:00': [1378.0, 1377.1],
#   '2018-07-10 12:13:00': [1391.0, 1389.0],
#   '2018-07-10 12:34:00': [1387.35, 1386.4],
#   '2018-07-10 14:46:00': [1370.05, 1369.7],
#   '2018-07-11 11:06:00': [1359.45, 1358.55],
#   '2018-07-11 11:25:00': [1356.0, 1354.4],
#   '2018-07-12 12:40:00': [1365.0, 1362.0],
#   '2018-07-12 12:43:00': [1360.45, 1357.1],
#   '2018-07-12 13:30:00': [1350.05, 1348.0],
#   '2018-07-13 11:15:00': [1335.15, 1334.45],
#   '2018-07-13 14:28:00': [1344.5, 1343.75],
#   '2018-07-13 15:23:00': [1335.15, 1333.65],
#   '2018-07-16 12:57:00': [1314.3, 1312.4],
#   '2018-07-16 14:33:00': [1288.0, 1286.0],
#   '2018-07-16 14:49:00': [1276.05, 1274.8],
#   '2018-07-17 10:54:00': [1271.0, 1269.9],
#   '2018-07-18 12:13:00': [1290.0, 1289.0],
#   '2018-07-18 12:20:00': [1284.25, 1279.7],
#   '2018-07-18 14:04:00': [1290.0, 1288.7],
#   '2018-07-25 10:40:00': [1460.0, 1456.75],
#   '2018-07-25 15:19:00': [1465.5, 1463.95],
#   '2018-07-30 14:31:00': [1505.7, 1504.95],
#   '2018-07-31 14:04:00': [1514.65, 1514.6],
#   '2018-08-01 11:31:00': [1552.0, 1545.4],
#   '2018-08-01 12:47:00': [1540.05, 1539.0],
#   '2018-08-01 13:23:00': [1534.6, 1534.0],
#   '2018-08-02 13:03:00': [1538.5, 1537.55],
#   '2018-08-02 14:14:00': [1536.0, 1535.0],
#   '2018-08-03 09:29:00': [1532.0, 1530.65],
#   '2018-08-03 12:48:00': [1534.35, 1530.8],
#   '2018-08-03 15:10:00': [1522.35, 1521.05],
#   '2018-08-03 15:16:00': [1515.0, 1514.8],
#   '2018-08-07 13:41:00': [1541.9, 1541.3],
#   '2018-08-07 15:18:00': [1539.0, 1537.35],
#   '2018-08-08 14:35:00': [1538.25, 1537.05],
#   '2018-08-08 15:10:00': [1536.5, 1535.0],
#   '2018-08-09 11:37:00': [1561.2, 1560.95],
#   '2018-08-10 11:00:00': [1563.0, 1562.35],
#   '2018-08-10 11:07:00': [1560.0, 1554.85],
#   '2018-08-10 13:23:00': [1548.0, 1547.5],
#   '2018-08-13 13:29:00': [1521.15, 1520.1],
#   '2018-08-16 09:30:00': [1540.35, 1537.9],
#   '2018-08-20 10:03:00': [1610.95, 1608.4],
#   '2018-08-20 11:13:00': [1601.1, 1599.4],
#   '2018-08-21 13:31:00': [1639.95, 1638.1],
#   '2018-08-24 12:29:00': [1638.2, 1636.25],
#   '2018-08-28 09:45:00': [1652.05, 1649.85],
#   '2018-08-28 11:14:00': [1637.9, 1637.0],
#   '2018-08-28 14:01:00': [1628.35, 1624.6],
#   '2018-08-29 09:28:00': [1618.0, 1615.95],
#   '2018-08-29 14:44:00': [1615.35, 1613.75],
#   '2018-08-30 12:31:00': [1616.1, 1615.75],
#   '2018-09-04 10:51:00': [1632.1, 1628.9],
#   '2018-09-04 13:44:00': [1612.45, 1610.8],
#   '2018-09-05 11:59:00': [1541.1, 1539.6],
#   '2018-09-06 11:10:00': [1566.05, 1563.8],
#   '2018-09-06 11:45:00': [1555.0, 1550.35],
#   '2018-09-10 10:46:00': [1547.45, 1545.75],
#   '2018-09-10 11:52:00': [1532.25, 1530.95],
#   '2018-09-10 14:11:00': [1534.0, 1533.95],
#   '2018-09-10 14:48:00': [1530.0, 1528.0],
#   '2018-09-11 13:59:00': [1535.0, 1534.1],
#   '2018-09-17 12:35:00': [1606.0, 1604.2],
#   '2018-09-18 10:24:00': [1570.35, 1568.45],
#   '2018-09-18 13:23:00': [1571.05, 1570.45],
#   '2018-09-21 09:48:00': [1568.1, 1563.8],
#   '2018-09-21 10:38:00': [1546.0, 1545.6],
#   '2018-09-21 13:02:00': [1511.15, 1499.95],
#   '2018-09-21 13:07:00': [1485.85, 1464.15],
#   '2018-09-28 13:50:00': [1550.2, 1549.95],
#   '2018-10-01 12:25:00': [1533.0, 1530.1],
#   '2018-10-01 14:52:00': [1557.4, 1557.35],
#   '2018-10-03 09:34:00': [1549.75, 1547.6],
#   '2018-10-03 10:16:00': [1535.0, 1533.95],
#   '2018-10-03 13:56:00': [1528.45, 1525.45],
#   '2018-10-08 12:40:00': [1467.0, 1465.15],
#   '2018-10-08 14:26:00': [1451.65, 1448.75],
#   '2018-10-09 10:42:00': [1478.0, 1477.05],
#   '2018-10-09 10:53:00': [1468.5, 1466.9],
#   '2018-10-10 11:42:00': [1467.55, 1465.0],
#   '2018-10-10 11:53:00': [1459.1, 1458.0],
#   '2018-10-11 13:29:00': [1421.55, 1420.7],
#   '2018-10-12 13:38:00': [1495.1, 1492.9],
#   '2018-10-16 12:33:00': [1557.1, 1555.0],
#   '2018-10-17 12:34:00': [1564.6, 1563.25],
#   '2018-10-17 14:33:00': [1553.05, 1547.65],
#   '2018-10-17 15:21:00': [1539.35, 1538.75],
#   '2018-10-19 12:41:00': [1438.15, 1437.4],
#   '2018-10-19 14:16:00': [1429.0, 1425.0],
#   '2018-10-19 14:21:00': [1421.5, 1414.6],
#   '2018-10-25 10:40:00': [1370.3, 1367.9],
#   '2018-10-26 10:59:00': [1346.5, 1346.0],
#   '2018-10-29 12:23:00': [1348.1, 1347.0],
#   '2018-10-30 12:17:00': [1352.8, 1350.8],
#   '2018-10-30 13:21:00': [1350.3, 1348.05],
#   '2018-11-05 14:30:00': [1469.25, 1467.0],
#   '2018-11-06 10:00:00': [1472.2, 1470.95],
#   '2018-11-06 13:02:00': [1459.05, 1457.5],
#   '2018-11-06 13:45:00': [1461.1, 1459.5],
#   '2018-11-06 13:54:00': [1456.0, 1454.3],
#   '2018-11-06 15:12:00': [1448.2, 1446.75],
#   '2018-11-09 14:58:00': [1466.0, 1464.95],
#   '2018-11-12 12:56:00': [1452.0, 1451.15],
#   '2018-11-12 13:07:00': [1450.25, 1449.25],
#   '2018-11-13 12:47:00': [1436.45, 1434.55],
#   '2018-11-16 13:10:00': [1516.0, 1513.7],
#   '2018-11-16 13:25:00': [1512.0, 1510.0],
#   '2018-11-19 10:29:00': [1503.25, 1500.0],
#   '2018-11-19 11:41:00': [1488.8, 1485.4],
#   '2018-11-19 12:46:00': [1496.7, 1490.95],
#   '2018-11-20 14:04:00': [1484.55, 1483.75],
#   '2018-11-20 14:30:00': [1478.0, 1474.0],
#   '2018-11-22 10:15:00': [1483.0, 1480.55],
#   '2018-11-22 11:54:00': [1474.5, 1472.25],
#   '2018-11-22 12:40:00': [1466.7, 1465.75],
#   '2018-11-22 13:05:00': [1463.5, 1463.0],
#   '2018-11-22 14:51:00': [1465.75, 1465.0],
#   '2018-11-22 15:16:00': [1456.75, 1454.9],
#   '2018-11-26 12:15:00': [1439.6, 1437.2],
#   '2018-12-04 11:11:00': [1512.25, 1511.45],
#   '2018-12-04 14:30:00': [1513.6, 1512.15],
#   '2018-12-05 12:07:00': [1482.1, 1478.0],
#   '2018-12-05 14:24:00': [1474.0, 1470.65],
#   '2018-12-06 10:50:00': [1450.3, 1450.1],
#   '2018-12-06 10:59:00': [1447.0, 1446.4],
#   '2018-12-06 13:33:00': [1450.0, 1448.95],
#   '2018-12-06 14:27:00': [1432.7, 1430.7],
#   '2018-12-07 10:32:00': [1430.55, 1428.8],
#   '2018-12-07 12:53:00': [1437.0, 1434.0],
#   '2018-12-10 13:14:00': [1422.0, 1420.35],
#   '2018-12-10 14:39:00': [1416.0, 1414.6],
#   '2018-12-12 11:01:00': [1448.25, 1446.8],
#   '2018-12-13 12:14:00': [1487.6, 1486.75],
#   '2018-12-14 11:25:00': [1507.0, 1505.6],
#   '2018-12-17 13:42:00': [1502.5, 1501.5],
#   '2018-12-18 13:41:00': [1498.0, 1497.0],
#   '2018-12-18 14:51:00': [1497.0, 1496.5],
#   '2018-12-19 14:00:00': [1558.2, 1555.1],
#   '2018-12-20 14:39:00': [1538.1, 1536.75],
#   '2018-12-21 13:06:00': [1504.0, 1502.35],
#   '2018-12-21 13:20:00': [1498.1, 1497.95],
#   '2018-12-21 13:56:00': [1495.5, 1493.0],
#   '2018-12-24 12:08:00': [1475.5, 1475.0],
#   '2018-12-24 13:29:00': [1472.1, 1470.5],
#   '2018-12-24 13:55:00': [1468.05, 1468.0],
#   '2018-12-24 14:28:00': [1466.0, 1462.7],
#   '2018-12-24 14:35:00': [1456.95, 1456.0],
#   '2018-12-27 11:29:00': [1475.15, 1474.0],
#   '2018-12-27 13:48:00': [1470.05, 1470.0],
#   '2018-12-27 15:01:00': [1466.5, 1466.0],
#   '2018-12-28 12:41:00': [1484.0, 1481.75],
#   '2018-12-28 15:11:00': [1480.15, 1479.4],
#   '2019-01-01 11:12:00': [1491.75, 1489.9],
#   '2019-01-01 11:47:00': [1489.1, 1487.6],
#   '2019-01-01 12:07:00': [1486.1, 1486.0],
#   '2019-01-01 12:31:00': [1481.2, 1480.05],
#   '2019-01-01 13:36:00': [1478.1, 1477.3],
#   '2019-01-02 12:45:00': [1487.0, 1485.8],
#   '2019-01-02 15:15:00': [1485.75, 1484.7],
#   '2019-01-03 10:40:00': [1477.0, 1474.5],
#   '2019-01-03 11:07:00': [1464.75, 1464.6],
#   '2019-01-03 13:43:00': [1463.85, 1461.5],
#   '2019-01-04 14:15:00': [1458.5, 1457.05],
#   '2019-01-07 14:36:00': [1488.35, 1486.5],
#   '2019-01-07 14:42:00': [1484.0, 1482.4],
#   '2019-01-08 11:09:00': [1473.8, 1472.0],
#   '2019-01-08 14:45:00': [1479.5, 1477.0],
#   '2019-01-09 13:39:00': [1469.0, 1466.35],
#   '2019-01-10 12:50:00': [1453.0, 1451.85],
#   '2019-01-11 13:59:00': [1459.05, 1457.55],
#   '2019-01-14 14:34:00': [1455.1, 1454.3],
#   '2019-01-15 12:29:00': [1488.75, 1488.5],
#   '2019-01-15 13:10:00': [1486.65, 1485.4],
#   '2019-01-15 14:07:00': [1482.0, 1481.95],
#   '2019-01-16 11:07:00': [1485.0, 1483.45],
#   '2019-01-16 13:41:00': [1476.4, 1475.95],
#   '2019-01-17 09:48:00': [1476.1, 1474.3],
#   '2019-01-17 10:45:00': [1472.55, 1470.0],
#   '2019-01-17 11:59:00': [1468.0, 1467.55],
#   '2019-01-17 12:25:00': [1462.6, 1462.55],
#   '2019-01-18 13:06:00': [1444.0, 1442.55],
#   '2019-01-18 14:22:00': [1443.0, 1442.7],
#   '2019-01-21 10:45:00': [1438.1, 1437.0],
#   '2019-01-21 14:29:00': [1427.0, 1426.0],
#   '2019-01-22 10:49:00': [1422.95, 1421.45],
#   '2019-01-22 14:37:00': [1434.05, 1432.75],
#   '2019-01-23 13:11:00': [1440.05, 1439.0],
#   '2019-01-23 15:06:00': [1434.4, 1432.1],
#   '2019-01-24 11:10:00': [1427.0, 1426.0],
#   '2019-01-24 11:58:00': [1422.15, 1422.0],
#   '2019-01-24 13:42:00': [1414.5, 1414.0],
#   '2019-01-25 11:10:00': [1416.05, 1415.2],
#   '2019-01-25 12:11:00': [1411.5, 1409.0],
#   '2019-01-25 13:57:00': [1390.0, 1387.05],
#   '2019-01-25 14:18:00': [1377.0, 1376.5],
#   '2019-01-25 14:48:00': [1373.1, 1372.75],
#   '2019-01-28 10:38:00': [1350.0, 1348.55],
#   '2019-01-29 12:50:00': [1365.9, 1364.25],
#   '2019-01-30 10:00:00': [1350.5, 1348.6],
#   '2019-02-05 13:02:00': [1425.25, 1424.8],
#   '2019-02-05 13:42:00': [1423.75, 1422.55],
#   '2019-02-05 13:52:00': [1416.0, 1408.0],
#   '2019-02-06 10:42:00': [1390.95, 1387.95],
#   '2019-02-06 12:41:00': [1382.7, 1380.0],
#   '2019-02-08 10:24:00': [1421.2, 1419.95],
#   '2019-02-08 11:42:00': [1410.25, 1409.55],
#   '2019-02-08 15:10:00': [1393.9, 1393.0],
#   '2019-02-11 11:16:00': [1384.0, 1381.0],
#   '2019-02-11 11:26:00': [1377.05, 1377.0],
#   '2019-02-11 14:53:00': [1376.0, 1374.5],
#   '2019-02-12 11:36:00': [1380.0, 1377.15],
#   '2019-02-12 12:53:00': [1376.9, 1375.95],
#   '2019-02-13 10:09:00': [1376.0, 1373.75],
#   '2019-02-14 11:36:00': [1360.0, 1359.35],
#   '2019-02-14 11:45:00': [1353.2, 1352.5],
#   '2019-02-14 12:50:00': [1353.0, 1352.65],
#   '2019-02-15 11:09:00': [1351.25, 1351.2],
#   '2019-02-18 11:38:00': [1340.6, 1339.0],
#   '2019-02-18 12:16:00': [1335.0, 1334.0],
#   '2019-02-18 13:12:00': [1339.05, 1336.7],
#   '2019-02-18 15:16:00': [1331.5, 1331.0],
#   '2019-02-21 14:35:00': [1391.0, 1389.9],
#   '2019-02-26 09:55:00': [1359.5, 1359.05],
#   '2019-02-27 11:25:00': [1445.05, 1444.3],
#   '2019-02-28 11:11:00': [1431.35, 1429.45],
#   '2019-02-28 13:47:00': [1420.05, 1419.45],
#   '2019-03-05 12:41:00': [1498.0, 1496.9],
#   '2019-03-06 12:42:00': [1550.9, 1550.0],
#   '2019-03-07 12:21:00': [1550.95, 1549.2],
#   '2019-03-08 11:55:00': [1565.05, 1564.15],
#   '2019-03-11 09:52:00': [1575.55, 1572.8],
#   '2019-03-12 13:40:00': [1601.6, 1599.95],
#   '2019-03-12 15:13:00': [1593.95, 1592.1],
#   '2019-03-14 12:15:00': [1545.05, 1542.2],
#   '2019-03-14 13:02:00': [1541.0, 1539.85],
#   '2019-03-15 10:06:00': [1550.0, 1547.75],
#   '2019-03-15 12:00:00': [1538.0, 1537.0],
#   '2019-03-15 12:40:00': [1535.15, 1533.7],
#   '2019-03-15 12:47:00': [1532.0, 1531.5],
#   '2019-03-18 10:30:00': [1550.25, 1547.5],
#   '2019-03-19 13:32:00': [1567.5, 1566.3],
#   '2019-03-20 10:57:00': [1562.8, 1561.8],
#   '2019-03-20 12:01:00': [1559.6, 1557.3],
#   '2019-03-22 14:01:00': [1586.35, 1585.95],
#   '2019-03-25 11:33:00': [1570.0, 1568.2],
#   '2019-03-25 14:51:00': [1573.7, 1572.4],
#   '2019-03-26 10:00:00': [1580.0, 1578.5],
#   '2019-03-26 10:26:00': [1576.35, 1573.65],
#   '2019-03-26 10:41:00': [1567.4, 1565.75],
#   '2019-03-26 13:04:00': [1559.5, 1559.4],
#   '2019-03-27 10:50:00': [1608.0, 1606.55],
#   '2019-03-27 11:09:00': [1602.65, 1602.0],
#   '2019-03-27 11:38:00': [1583.05, 1583.0],
#   '2019-03-28 11:19:00': [1605.15, 1603.4],
#   '2019-03-28 12:32:00': [1602.45, 1599.3],
#   '2019-03-28 13:56:00': [1598.95, 1595.0],
#   '2019-04-01 10:37:00': [1681.75, 1677.6],
#   '2019-04-01 13:13:00': [1672.0, 1668.9],
#   '2019-04-01 14:33:00': [1666.0, 1661.25],
#   '2019-04-01 15:01:00': [1655.65, 1653.8],
#   '2019-04-02 11:20:00': [1652.0, 1651.7],
#   '2019-04-02 11:39:00': [1648.0, 1646.3],
#   '2019-04-02 11:47:00': [1639.4, 1638.1],
#   '2019-04-02 13:03:00': [1641.0, 1636.95],
#   '2019-04-02 14:06:00': [1631.45, 1630.0],
#   '2019-04-02 14:10:00': [1627.35, 1626.6],
#   '2019-04-04 13:52:00': [1635.9, 1635.55],
#   '2019-04-04 14:16:00': [1625.1, 1624.6],
#   '2019-04-11 10:02:00': [1641.05, 1639.9],
#   '2019-04-15 14:16:00': [1665.05, 1665.0],
#   '2019-04-16 13:16:00': [1698.8, 1698.15],
#   '2019-04-18 13:53:00': [1688.35, 1687.9],
#   '2019-04-22 12:47:00': [1682.1, 1682.05],
#   '2019-04-22 14:04:00': [1678.2, 1675.9],
#   '2019-04-23 10:03:00': [1670.65, 1669.8],
#   '2019-04-23 10:48:00': [1671.4, 1669.85],
#   '2019-04-23 12:12:00': [1673.0, 1672.7],
#   '2019-04-23 14:37:00': [1670.25, 1669.35],
#   '2019-04-23 15:11:00': [1666.15, 1661.2],
#   '2019-04-24 11:44:00': [1582.0, 1577.45],
#   '2019-04-25 13:33:00': [1640.6, 1638.75],
#   '2019-04-25 14:53:00': [1630.0, 1627.25],
#   '2019-04-30 09:43:00': [1626.05, 1624.15],
#   '2019-04-30 11:53:00': [1606.3, 1605.7],
#   '2019-05-02 11:00:00': [1631.35, 1631.0],
#   '2019-05-07 13:59:00': [1624.9, 1623.0],
#   '2019-05-07 14:58:00': [1605.1, 1602.7],
#   '2019-05-09 12:19:00': [1586.0, 1584.0],
#   '2019-05-09 13:43:00': [1579.1, 1579.0],
#   '2019-05-10 12:47:00': [1603.95, 1603.2],
#   '2019-05-20 14:26:00': [1665.9, 1663.6],
#   '2019-05-21 11:47:00': [1670.8, 1669.8],
#   '2019-05-21 12:39:00': [1662.25, 1661.1],
#   '2019-05-21 12:57:00': [1652.0, 1648.75],
#   '2019-05-22 10:12:00': [1618.25, 1617.5],
#   '2019-05-22 14:31:00': [1618.05, 1615.95],
#   '2019-05-29 14:36:00': [1718.35, 1718.0],
#   '2019-05-30 10:41:00': [1721.55, 1720.35],
#   '2019-05-30 12:38:00': [1714.15, 1712.5],
#   '2019-05-30 13:56:00': [1708.05, 1706.25],
#   '2019-05-31 13:22:00': [1673.0, 1672.5],
#   '2019-06-03 13:43:00': [1701.5, 1701.4],
#   '2019-06-04 10:31:00': [1706.15, 1704.0],
#   '2019-06-04 11:02:00': [1688.55, 1685.75],
#   '2019-06-04 12:10:00': [1670.1, 1669.6],
#   '2019-06-04 12:21:00': [1661.8, 1659.0],
#   '2019-06-04 14:09:00': [1661.0, 1660.0],
#   '2019-06-06 10:55:00': [1612.0, 1611.35],
#   '2019-06-06 12:28:00': [1607.0, 1605.75],
#   '2019-06-06 13:13:00': [1600.1, 1599.0],
#   '2019-06-06 13:27:00': [1595.7, 1595.3],
#   '2019-06-06 13:37:00': [1590.1, 1589.1],
#   '2019-06-06 15:16:00': [1590.0, 1589.8],
#   '2019-06-07 10:42:00': [1598.3, 1594.05],
#   '2019-06-07 14:56:00': [1593.7, 1593.65],
#   '2019-06-17 10:46:00': [1525.0, 1523.9],
#   '2019-06-17 12:44:00': [1525.1, 1525.0],
#   '2019-06-18 14:49:00': [1519.3, 1517.5],
#   '2019-06-19 09:44:00': [1526.05, 1524.2],
#   '2019-06-19 11:05:00': [1513.05, 1510.85],
#   '2019-06-19 11:15:00': [1509.0, 1508.15],
#   '2019-06-20 13:12:00': [1500.1, 1499.7],
#   '2019-06-21 12:38:00': [1525.5, 1520.95],
#   '2019-06-21 15:03:00': [1530.45, 1530.05],
#   '2019-06-24 12:45:00': [1538.65, 1536.35],
#   '2019-06-26 10:50:00': [1571.0, 1570.65],
#   '2019-06-27 14:04:00': [1576.0, 1574.4],
#   '2019-06-28 13:11:00': [1576.0, 1573.4],
#   '2019-07-01 12:17:00': [1561.2, 1561.0],
#   '2019-07-01 12:31:00': [1557.2, 1556.25],
#   '2019-07-03 13:14:00': [1567.45, 1566.7],
#   '2019-07-03 13:18:00': [1566.0, 1564.4],
#   '2019-07-04 10:01:00': [1566.1, 1565.85],
#   '2019-07-05 10:45:00': [1590.05, 1588.9],
#   '2019-07-05 11:25:00': [1588.0, 1586.8],
#   '2019-07-05 11:45:00': [1581.05, 1579.95],
#   '2019-07-05 15:01:00': [1550.6, 1547.45],
#   '2019-07-08 10:35:00': [1523.0, 1519.3],
#   '2019-07-10 12:39:00': [1546.55, 1545.85],
#   '2019-07-10 15:16:00': [1536.5, 1535.8],
#   '2019-07-11 13:32:00': [1550.5, 1549.75],
#   '2019-07-12 11:13:00': [1568.25, 1567.0],
#   '2019-07-12 14:39:00': [1573.0, 1572.6],
#   '2019-07-15 10:07:00': [1571.25, 1568.25],
#   '2019-07-15 10:15:00': [1566.0, 1563.95],
#   '2019-07-17 12:59:00': [1603.05, 1602.8],
#   '2019-07-17 13:11:00': [1601.0, 1598.8],
#   '2019-07-17 13:19:00': [1593.0, 1591.0],
#   '2019-07-17 15:09:00': [1595.3, 1595.25],
#   '2019-07-17 15:12:00': [1591.1, 1589.6],
#   '2019-07-18 10:23:00': [1596.5, 1594.2],
#   '2019-07-18 15:11:00': [1568.0, 1566.35],
#   '2019-07-19 10:00:00': [1612.0, 1610.9],
#   '2019-07-19 13:16:00': [1587.0, 1583.55],
#   '2019-07-19 14:29:00': [1580.65, 1578.85],
#   '2019-07-19 14:45:00': [1567.85, 1560.0],
#   '2019-07-19 15:11:00': [1541.0, 1536.0],
#   '2019-07-22 13:20:00': [1567.85, 1567.2],
#   '2019-07-23 10:46:00': [1589.0, 1588.65],
#   '2019-07-23 13:15:00': [1590.15, 1589.5],
#   '2019-07-23 14:10:00': [1584.95, 1583.95],
#   '2019-07-23 14:28:00': [1581.45, 1580.7],
#   '2019-07-24 10:36:00': [1571.5, 1569.9],
#   '2019-07-24 12:33:00': [1571.25, 1571.0],
#   '2019-07-24 14:45:00': [1570.65, 1569.95],
#   '2019-07-25 15:03:00': [1577.25, 1576.4],
#   '2019-07-29 09:39:00': [1576.85, 1575.6],
#   '2019-07-29 11:28:00': [1555.1, 1553.45],
#   '2019-07-29 13:29:00': [1546.5, 1545.0],
#   '2019-07-29 14:36:00': [1537.05, 1535.6],
#   '2019-07-30 15:16:00': [1540.35, 1538.25],
#   '2019-08-01 13:03:00': [1537.8, 1537.0],
#   '2019-08-01 13:35:00': [1536.35, 1533.75],
#   '2019-08-01 14:34:00': [1516.7, 1516.05],
#   '2019-08-07 15:00:00': [1560.1, 1557.85],
#   '2019-08-07 15:10:00': [1555.0, 1553.95],
#   '2019-08-08 12:11:00': [1579.0, 1577.9],
#   '2019-08-08 13:02:00': [1575.15, 1573.25],
#   '2019-08-08 14:12:00': [1566.05, 1565.05],
#   '2019-08-08 14:15:00': [1559.05, 1554.35],
#   '2019-08-09 13:42:00': [1581.0, 1578.85],
#   '2019-08-13 12:22:00': [1565.0, 1562.9],
#   '2019-08-13 14:28:00': [1557.5, 1555.95],
#   '2019-08-19 11:23:00': [1603.1, 1601.45],
#   '2019-08-19 13:33:00': [1599.15, 1598.45],
#   '2019-08-19 14:29:00': [1593.75, 1592.25],
#   '2019-08-19 15:11:00': [1589.15, 1587.6],
#   '2019-08-20 09:52:00': [1553.8, 1551.9],
#   '2019-08-20 10:43:00': [1547.25, 1544.0],
#   '2019-08-20 12:17:00': [1542.35, 1541.85],
#   '2019-08-21 12:00:00': [1515.95, 1513.0],
#   '2019-08-21 14:13:00': [1488.75, 1487.75],
#   '2019-08-22 10:34:00': [1467.5, 1463.8],
#   '2019-08-22 14:55:00': [1471.5, 1469.65],
#   '2019-08-22 15:00:00': [1467.0, 1463.1],
#   '2019-08-23 11:13:00': [1456.1, 1455.65],
#   '2019-08-23 12:00:00': [1460.65, 1458.5],
#   '2019-08-27 12:34:00': [1532.7, 1532.0],
#   '2019-08-27 12:51:00': [1530.7, 1528.7],
#   '2019-08-27 15:02:00': [1539.1, 1537.5],
#   '2019-08-28 12:30:00': [1540.55, 1538.75],
#   '2019-08-28 12:39:00': [1535.9, 1533.3],
#   '2019-08-28 14:14:00': [1529.7, 1526.95],
#   '2019-08-28 14:32:00': [1524.5, 1523.0],
#   '2019-08-29 10:18:00': [1538.0, 1535.4],
#   '2019-08-29 11:58:00': [1515.25, 1513.0],
#   '2019-08-29 15:06:00': [1503.45, 1501.35],
#   '2019-08-29 15:11:00': [1498.0, 1497.05],
#   '2019-08-30 10:16:00': [1497.0, 1495.15],
#   '2019-08-30 11:10:00': [1490.2, 1487.85],
#   '2019-09-03 10:29:00': [1473.0, 1472.1],
#   '2019-09-03 11:17:00': [1468.4, 1468.1],
#   '2019-09-03 11:45:00': [1461.0, 1459.8],
#   '2019-09-03 15:16:00': [1454.6, 1452.05],
#   '2019-09-03 15:23:00': [1445.0, 1443.4],
#   '2019-09-04 11:19:00': [1439.05, 1437.3],
#   '2019-09-05 13:49:00': [1459.0, 1457.9],
#   '2019-09-06 11:57:00': [1468.3, 1466.3],
#   '2019-09-06 14:13:00': [1463.2, 1462.15],
#   '2019-09-06 14:18:00': [1460.0, 1459.0],
#   '2019-09-09 13:45:00': [1476.0, 1473.8],
#   '2019-09-11 10:04:00': [1483.8, 1482.85],
#   '2019-09-11 11:14:00': [1480.5, 1479.15],
#   '2019-09-12 14:05:00': [1491.55, 1490.85],
#   '2019-09-16 11:57:00': [1530.3, 1529.85],
#   '2019-09-17 13:26:00': [1500.05, 1498.05],
#   '2019-09-18 13:04:00': [1495.0, 1494.85],
#   '2019-09-19 10:28:00': [1488.7, 1487.6],
#   '2019-09-19 14:32:00': [1471.15, 1469.35],
#   '2019-09-23 13:24:00': [1684.0, 1679.05],
#   '2019-09-23 13:37:00': [1675.65, 1672.45],
#   '2019-09-24 13:23:00': [1636.7, 1635.05],
#   '2019-09-25 09:43:00': [1632.6, 1631.3],
#   '2019-09-25 11:24:00': [1643.0, 1640.0],
#   '2019-09-25 13:06:00': [1638.8, 1636.3],
#   '2019-09-26 10:30:00': [1640.0, 1638.3],
#   '2019-09-26 11:52:00': [1635.0, 1634.8],
#   '2019-09-26 15:06:00': [1642.5, 1642.0],
#   '2019-09-30 09:40:00': [1620.55, 1619.7],
#   '2019-09-30 10:18:00': [1621.15, 1619.05],
#   '2019-09-30 14:50:00': [1601.0, 1598.7],
#   '2019-10-01 09:57:00': [1604.55, 1603.5],
#   '2019-10-01 13:15:00': [1580.1, 1578.9],
#   '2019-10-01 13:31:00': [1571.1, 1568.85],
#   '2019-10-03 12:40:00': [1559.1, 1557.1],
#   '2019-10-04 10:52:00': [1535.1, 1534.9],
#   '2019-10-04 13:30:00': [1503.5, 1502.95],
#   '2019-10-04 14:59:00': [1497.5, 1497.2],
#   '2019-10-07 11:08:00': [1460.95, 1460.45],
#   '2019-10-09 10:29:00': [1427.55, 1422.25],
#   '2019-10-10 13:43:00': [1425.0, 1423.5],
#   '2019-10-11 10:00:00': [1438.05, 1437.3],
#   '2019-10-11 10:42:00': [1434.0, 1430.2],
#   '2019-10-11 14:31:00': [1436.8, 1436.75],
#   '2019-10-14 11:51:00': [1465.95, 1462.85],
#   '2019-10-14 14:51:00': [1478.05, 1474.55],
#   '2019-10-16 11:29:00': [1529.5, 1523.95],
#   '2019-10-16 14:35:00': [1521.2, 1519.6],
#   '2019-10-17 12:03:00': [1522.15, 1521.95],
#   '2019-10-17 12:26:00': [1517.25, 1512.1],
#   '2019-10-23 12:07:00': [1540.25, 1539.85],
#   '2019-10-23 12:26:00': [1536.0, 1535.5],
#   '2019-10-24 10:30:00': [1525.0, 1524.3],
#   '2019-10-25 12:48:00': [1519.8, 1517.0],
#   '2019-10-29 10:04:00': [1521.4, 1519.05],
#   '2019-10-30 13:17:00': [1547.35, 1546.9],
#   '2019-10-30 14:06:00': [1545.0, 1542.15],
#   '2019-10-30 14:31:00': [1534.0, 1533.25],
#   '2019-10-30 14:40:00': [1525.85, 1523.7],
#   '2019-11-01 12:55:00': [1564.0, 1563.2],
#   '2019-11-01 13:10:00': [1551.1, 1549.0],
#   '2019-11-04 13:12:00': [1573.85, 1573.0],
#   '2019-11-04 13:51:00': [1569.5, 1569.3],
#   '2019-11-04 15:11:00': [1567.0, 1566.55],
#   '2019-11-05 10:48:00': [1553.95, 1553.15],
#   '2019-11-05 11:17:00': [1549.2, 1547.65],
#   '2019-11-05 11:35:00': [1539.25, 1536.35],
#   '2019-11-05 13:12:00': [1527.15, 1525.75],
#   '2019-11-05 15:11:00': [1518.35, 1515.25],
#   '2019-11-06 13:52:00': [1525.0, 1523.45],
#   '2019-11-07 12:30:00': [1540.1, 1538.7],
#   '2019-11-07 12:35:00': [1535.6, 1534.3],
#   '2019-11-07 12:59:00': [1528.35, 1525.6],
#   '2019-11-08 11:25:00': [1527.9, 1525.35],
#   '2019-11-08 13:13:00': [1520.25, 1519.95],
#   '2019-11-11 11:50:00': [1510.0, 1504.9],
#   '2019-11-11 13:25:00': [1496.5, 1495.3],
#   '2019-11-13 15:16:00': [1493.15, 1492.7],
#   '2019-11-14 14:06:00': [1478.0, 1476.9],
#   '2019-11-15 12:45:00': [1488.05, 1487.15],
#   '2019-11-15 15:02:00': [1487.15, 1486.15],
#   '2019-11-18 11:30:00': [1500.0, 1498.0],
#   '2019-11-18 11:55:00': [1495.65, 1494.8],
#   '2019-11-19 11:29:00': [1495.75, 1490.3],
#   '2019-11-19 12:30:00': [1494.1, 1493.0],
#   '2019-11-19 12:53:00': [1488.75, 1486.6],
#   '2019-11-20 11:08:00': [1495.25, 1494.5],
#   '2019-11-20 12:38:00': [1494.35, 1491.6],
#   '2019-11-20 14:45:00': [1485.45, 1485.0],
#   '2019-11-21 12:39:00': [1480.1, 1479.1],
#   '2019-11-22 10:15:00': [1474.8, 1471.3],
#   '2019-11-26 10:58:00': [1485.2, 1482.35],
#   '2019-11-26 13:13:00': [1484.5, 1484.0],
#   '2019-11-27 15:18:00': [1502.0, 1501.75],
#   '2019-11-29 10:47:00': [1524.2, 1524.0],
#   '2019-12-02 13:11:00': [1540.3, 1540.05],
#   '2019-12-02 13:22:00': [1537.0, 1536.0],
#   '2019-12-02 14:39:00': [1531.0, 1529.5],
#   '2019-12-03 10:44:00': [1523.0, 1522.7],
#   '2019-12-03 15:09:00': [1510.05, 1509.5],
#   '2019-12-05 10:28:00': [1511.4, 1511.0],
#   '2019-12-05 11:46:00': [1501.0, 1500.2],
#   '2019-12-05 14:01:00': [1498.25, 1496.3],
#   '2019-12-06 13:53:00': [1481.55, 1481.05],
#   '2019-12-06 14:23:00': [1475.0, 1469.2],
#   '2019-12-09 12:45:00': [1486.5, 1484.95],
#   '2019-12-10 12:13:00': [1462.7, 1460.9],
#   '2019-12-10 15:07:00': [1439.15, 1437.0],
#   '2019-12-11 11:28:00': [1425.25, 1423.6],
#   '2019-12-11 11:41:00': [1420.3, 1419.25],
#   '2019-12-11 12:17:00': [1410.05, 1410.0],
#   '2019-12-11 12:49:00': [1405.6, 1405.0],
#   '2019-12-11 12:57:00': [1403.05, 1403.0],
#   '2019-12-12 10:39:00': [1422.0, 1420.9],
#   '2019-12-12 12:33:00': [1424.7, 1422.05],
#   '2019-12-13 10:52:00': [1438.2, 1437.0],
#   '2019-12-16 11:10:00': [1440.35, 1438.85],
#   '2019-12-16 12:29:00': [1430.1, 1429.5],
#   '2019-12-16 15:01:00': [1439.05, 1438.8],
#   '2019-12-18 13:03:00': [1460.0, 1458.45],
#   '2019-12-19 11:52:00': [1454.15, 1454.05],
#   '2019-12-19 12:40:00': [1452.5, 1451.55],
#   '2019-12-19 13:45:00': [1447.0, 1445.65],
#   '2019-12-20 10:17:00': [1440.8, 1439.5],
#   '2019-12-27 13:08:00': [1458.25, 1457.4],
#   '2019-12-30 10:49:00': [1465.0, 1461.8],
#   '2019-12-31 10:59:00': [1448.5, 1447.6],
#   '2019-12-31 14:19:00': [1448.55, 1448.0],
#   '2019-12-31 15:00:00': [1446.25, 1446.05],
#   '2020-01-01 10:44:00': [1450.15, 1448.0],
#   '2020-01-01 12:41:00': [1443.1, 1443.0],
#   '2020-01-01 15:11:00': [1440.35, 1439.8],
#   '2020-01-02 13:50:00': [1494.0, 1493.9],
#   '2020-01-03 12:01:00': [1476.0, 1475.0],
#   '2020-01-03 14:13:00': [1471.95, 1470.9],
#   '2020-01-03 14:50:00': [1470.05, 1469.0],
#   '2020-01-06 09:30:00': [1453.5, 1451.35],
#   '2020-01-06 12:58:00': [1444.0, 1441.9],
#   '2020-01-06 14:06:00': [1442.7, 1441.4],
#   '2020-01-06 15:12:00': [1433.85, 1433.7],
#   '2020-01-07 11:11:00': [1446.0, 1444.2],
#   '2020-01-08 14:15:00': [1466.05, 1465.0],
#   '2020-01-09 11:10:00': [1482.35, 1481.2],
#   '2020-01-10 13:06:00': [1520.0, 1519.2],
#   '2020-01-13 12:06:00': [1522.5, 1521.05],
#   '2020-01-14 13:00:00': [1522.85, 1521.0],
#   '2020-01-14 13:48:00': [1517.6, 1516.8],
#   '2020-01-14 14:13:00': [1515.15, 1515.0],
#   '2020-01-15 12:06:00': [1494.6, 1494.4],
#   '2020-01-16 09:51:00': [1520.3, 1518.0],
#   '2020-01-16 10:25:00': [1512.5, 1509.8],
#   '2020-01-16 11:51:00': [1506.65, 1502.1],
#   '2020-01-16 14:21:00': [1505.5, 1504.9],
#   '2020-01-17 13:19:00': [1512.75, 1512.4],
#   '2020-01-20 09:55:00': [1509.1, 1507.95],
#   '2020-01-20 12:25:00': [1503.5, 1503.25],
#   '2020-01-20 14:26:00': [1499.3, 1497.55],
#   '2020-01-21 10:45:00': [1500.05, 1498.2],
#   '2020-01-21 15:10:00': [1496.75, 1493.4],
#   '2020-01-22 12:45:00': [1527.55, 1526.5],
#   '2020-01-22 14:05:00': [1531.3, 1530.85],
#   '2020-01-22 14:22:00': [1525.4, 1523.95],
#   '2020-01-23 11:56:00': [1539.05, 1538.1],
#   '2020-01-23 15:11:00': [1548.65, 1547.95],
#   '2020-01-24 13:01:00': [1556.05, 1554.6],
#   '2020-01-24 13:57:00': [1555.0, 1552.5],
#   '2020-01-27 12:33:00': [1565.9, 1565.0],
#   '2020-01-27 14:05:00': [1560.0, 1559.35],
#   '2020-01-28 11:58:00': [1570.55, 1568.6],
#   '2020-01-28 14:52:00': [1555.05, 1554.2],
#   '2020-01-29 10:10:00': [1564.5, 1562.35],
#   '2020-01-29 10:15:00': [1560.1, 1558.8],
#   '2020-01-29 13:37:00': [1565.15, 1563.95],
#   '2020-01-29 14:37:00': [1564.45, 1563.2],
#   '2020-01-29 15:00:00': [1558.6, 1558.1],
#   '2020-01-29 15:18:00': [1550.0, 1548.3],
#   '2020-01-30 12:21:00': [1537.25, 1536.5],
#   '2020-01-30 13:55:00': [1534.05, 1532.5],
#   '2020-01-31 11:44:00': [1514.4, 1511.8],
#   '2020-01-31 14:45:00': [1515.0, 1513.75],
#   '2020-01-31 15:03:00': [1507.5, 1503.5],
#   '2020-02-01 11:21:00': [1520.7, 1519.65],
#   '2020-02-01 12:04:00': [1511.4, 1508.6],
#   '2020-02-01 12:53:00': [1506.0, 1503.2],
#   '2020-02-01 13:28:00': [1484.6, 1476.1],
#   '2020-02-05 11:30:00': [1477.2, 1476.5],
#   '2020-02-05 12:49:00': [1467.35, 1465.65],
#   '2020-02-06 10:10:00': [1468.05, 1466.85],
#   '2020-02-10 09:56:00': [1462.0, 1461.75],
#   '2020-02-10 11:00:00': [1456.6, 1455.9],
#   '2020-02-10 13:22:00': [1466.5, 1466.0],
#   '2020-02-10 15:11:00': [1452.7, 1451.75],
#   '2020-02-11 12:46:00': [1447.0, 1444.75],
#   '2020-02-13 09:55:00': [1448.0, 1445.8],
#   '2020-02-13 14:43:00': [1452.0, 1450.0],
#   '2020-02-14 11:45:00': [1445.0, 1443.0],
#   '2020-02-14 14:39:00': [1440.1, 1439.45],
#   '2020-02-17 09:30:00': [1425.8, 1421.35],
#   '2020-02-17 13:00:00': [1426.2, 1424.1],
#   '2020-02-19 10:06:00': [1427.5, 1427.25],
#   '2020-02-20 10:42:00': [1444.45, 1443.45],
#   '2020-02-20 12:39:00': [1441.0, 1440.2],
#   '2020-02-20 13:17:00': [1430.55, 1430.5],
#   '2020-02-20 14:43:00': [1425.75, 1424.95],
#   '2020-02-24 12:48:00': [1432.0, 1431.9],
#   '2020-02-24 14:25:00': [1421.05, 1419.25],
#   '2020-02-24 14:33:00': [1415.05, 1414.75],
#   '2020-02-24 14:54:00': [1410.7, 1409.6],
#   '2020-03-02 14:24:00': [1341.0, 1340.6],
#   '2020-03-02 14:57:00': [1323.8, 1321.1],
#   '2020-03-03 11:39:00': [1300.1, 1299.35],
#   '2020-03-04 12:54:00': [1325.0, 1321.3],
#   '2020-03-05 12:40:00': [1330.6, 1328.9],
#   '2020-03-05 13:18:00': [1323.0, 1322.9],
#   '2020-03-05 13:26:00': [1320.2, 1319.0],
#   '2020-03-05 13:44:00': [1310.0, 1308.3],
#   '2020-03-06 14:10:00': [1293.3, 1293.0],
#   '2020-03-09 12:49:00': [1274.05, 1267.4],
#   '2020-03-09 12:53:00': [1258.3, 1256.65],
#   '2020-03-09 13:29:00': [1240.2, 1236.35],
#   '2020-03-12 10:47:00': [1123.4, 1119.75],
#   '2020-03-12 10:56:00': [1108.0, 1099.7],
#   '2020-03-12 14:33:00': [1136.2, 1134.35],
#   '2020-03-18 12:09:00': [1100.35, 1097.9],
#   '2020-03-18 14:12:00': [1081.15, 1077.95],
#   '2020-03-19 15:10:00': [1086.35, 1085.0],
#   '2020-03-20 12:34:00': [1132.25, 1132.1],
#   '2020-03-20 13:50:00': [1119.0, 1117.35],
#   '2020-03-20 13:53:00': [1111.0, 1109.0],
#   '2020-03-23 09:35:00': [1030.0, 1022.8],
#   '2020-03-23 15:16:00': [995.25, 993.0],
#   '2020-03-24 09:49:00': [977.25, 968.65],
#   '2020-03-24 10:57:00': [956.0, 955.7],
#   '2020-03-25 10:15:00': [901.2, 900.0],
#   '2020-03-26 10:48:00': [930.0, 924.2],
#   '2020-03-30 14:47:00': [940.5, 940.35],
#   '2020-04-01 14:51:00': [956.15, 955.2],
#   '2020-04-07 11:46:00': [1002.5, 999.15],
#   '2020-04-07 12:36:00': [989.5, 986.7],
#   '2020-04-08 11:46:00': [969.2, 964.15],
#   '2020-04-08 12:25:00': [953.8, 952.0],
#   '2020-04-09 10:06:00': [957.45, 956.0],
#   '2020-04-09 14:25:00': [1002.1, 998.0],
#   '2020-04-13 13:42:00': [1071.45, 1067.85],
#   '2020-04-15 13:46:00': [1149.8, 1145.6],
#   '2020-04-15 14:09:00': [1129.1, 1128.4],
#   '2020-04-16 10:33:00': [1147.0, 1146.7],
#   '2020-04-17 10:45:00': [1160.3, 1158.5],
#   '2020-04-17 11:56:00': [1156.3, 1155.0],
#   '2020-04-20 13:46:00': [1192.65, 1191.4],
#   '2020-04-21 12:18:00': [1148.95, 1147.15],
#   '2020-04-22 09:41:00': [1135.35, 1134.2],
#   '2020-04-23 11:08:00': [1216.8, 1214.2],
#   '2020-04-23 12:35:00': [1206.0, 1201.05],
#   '2020-04-23 13:05:00': [1194.65, 1194.0],
#   '2020-04-23 13:15:00': [1187.25, 1186.9],
#   '2020-04-27 12:31:00': [1145.0, 1142.35],
#   '2020-04-28 11:50:00': [1145.85, 1138.75],
#   '2020-04-28 12:23:00': [1132.7, 1131.0],
#   '2020-04-28 13:04:00': [1130.55, 1128.75],
#   '2020-04-28 14:20:00': [1130.7, 1127.6],
#   '2020-04-28 14:37:00': [1126.0, 1123.55],
#   '2020-04-29 14:17:00': [1125.0, 1122.45],
#   '2020-05-07 13:25:00': [1165.1, 1165.0],
#   '2020-05-07 14:45:00': [1157.75, 1156.85],
#   '2020-05-07 15:11:00': [1153.0, 1152.0],
#   '2020-05-08 11:02:00': [1155.0, 1152.0],
#   '2020-05-08 13:23:00': [1156.0, 1154.5],
#   '2020-05-08 14:41:00': [1150.0, 1149.85],
#   '2020-05-12 11:32:00': [1155.25, 1154.9],
#   '2020-05-12 12:15:00': [1151.55, 1150.65],
#   '2020-05-13 11:18:00': [1243.0, 1239.95],
#   '2020-05-13 12:24:00': [1233.0, 1230.0],
#   '2020-05-13 13:04:00': [1222.1, 1220.0],
#   '2020-05-15 10:36:00': [1190.0, 1189.0],
#   '2020-05-15 14:15:00': [1183.05, 1182.0],
#   '2020-05-18 10:59:00': [1157.35, 1155.15],
#   '2020-05-18 12:11:00': [1147.35, 1144.9],
#   '2020-05-18 15:00:00': [1141.0, 1139.35],
#   '2020-05-19 15:00:00': [1139.8, 1138.5],
#   '2020-05-26 11:20:00': [1255.0, 1252.2],
#   '2020-05-27 12:49:00': [1251.4, 1246.25],
#   '2020-05-28 11:18:00': [1259.5, 1259.0],
#   '2020-05-28 13:59:00': [1270.3, 1269.9],
#   '2020-05-29 13:40:00': [1281.3, 1279.8],
#   '2020-05-29 13:53:00': [1272.65, 1270.5],
#   '2020-06-01 10:35:00': [1303.0, 1301.55],
#   '2020-06-01 13:45:00': [1290.0, 1289.6],
#   '2020-06-01 15:25:00': [1280.05, 1279.5],
#   '2020-06-03 14:43:00': [1277.0, 1275.85],
#   '2020-06-09 13:59:00': [1269.75, 1269.4],
#   '2020-06-09 14:09:00': [1262.0, 1261.75],
#   '2020-06-09 14:33:00': [1255.85, 1255.0],
#   '2020-06-10 11:56:00': [1270.3, 1270.1],
#   '2020-06-11 10:49:00': [1270.5, 1269.95],
#   '2020-06-12 12:22:00': [1241.05, 1239.85],
#   '2020-06-15 10:59:00': [1246.05, 1246.0],
#   '2020-06-16 13:06:00': [1244.8, 1244.0],
#   '2020-06-16 13:15:00': [1235.35, 1231.15],
#   '2020-06-17 11:00:00': [1230.5, 1230.4],
#   '2020-06-17 13:33:00': [1238.15, 1237.85],
#   '2020-06-22 10:12:00': [1265.0, 1263.3],
#   '2020-06-22 15:04:00': [1261.7, 1260.2],
#   '2020-06-23 11:15:00': [1278.0, 1277.9],
#   '2020-06-23 13:50:00': [1297.1, 1294.85],
#   '2020-06-24 11:35:00': [1291.0, 1289.85],
#   '2020-06-24 12:34:00': [1283.75, 1283.7],
#   '2020-06-25 11:44:00': [1265.4, 1264.0],
#   '2020-06-26 10:17:00': [1261.65, 1260.0],
#   '2020-06-26 12:45:00': [1274.3, 1273.9],
#   '2020-06-30 14:17:00': [1323.7, 1322.5],
#   '2020-06-30 14:28:00': [1318.05, 1315.55],
#   '2020-07-01 10:19:00': [1322.05, 1320.35],
#   '2020-07-02 11:05:00': [1322.5, 1320.9],
#   '2020-07-02 12:34:00': [1325.0, 1322.7],
#   '2020-07-03 12:58:00': [1313.0, 1310.0],
#   '2020-07-09 13:05:00': [1322.35, 1322.3],
#   '2020-07-10 10:30:00': [1318.55, 1317.25],
#   '2020-07-10 14:25:00': [1310.0, 1309.3],
#   '2020-07-14 10:31:00': [1335.0, 1333.5],
#   '2020-07-14 12:49:00': [1305.0, 1302.75],
#   '2020-07-14 12:55:00': [1300.1, 1298.45],
#   '2020-07-15 12:04:00': [1300.0, 1298.3],
#   '2020-07-16 09:27:00': [1285.0, 1282.3],
#   '2020-07-16 11:07:00': [1260.1, 1256.35],
#   '2020-07-17 10:32:00': [1305.0, 1302.55],
#   '2020-07-20 13:32:00': [1322.0, 1321.55],
#   '2020-07-21 13:33:00': [1387.25, 1385.55],
#   '2020-07-23 10:59:00': [1363.0, 1362.95],
#   '2020-07-23 11:23:00': [1361.0, 1359.75],
#   '2020-07-23 12:45:00': [1361.0, 1360.45],
#   '2020-07-27 13:22:00': [1354.35, 1352.45],
#   '2020-07-28 11:59:00': [1385.0, 1383.3],
#   '2020-07-29 09:58:00': [1403.05, 1399.95],
#   '2020-07-29 12:17:00': [1403.5, 1402.15],
#   '2020-07-30 11:33:00': [1438.0, 1437.5],
#   '2020-07-31 10:35:00': [1425.55, 1420.35],
#   '2020-07-31 14:45:00': [1425.0, 1423.4],
#   '2020-08-03 12:06:00': [1443.05, 1442.3],
#   '2020-08-03 12:22:00': [1438.75, 1438.0],
#   '2020-08-03 13:36:00': [1433.5, 1429.8],
#   '2020-08-03 13:51:00': [1425.1, 1423.25],
#   '2020-08-03 15:03:00': [1416.15, 1414.9],
#   '2020-08-04 09:57:00': [1411.3, 1407.25],
#   '2020-08-05 11:31:00': [1418.0, 1416.95],
#   '2020-08-05 11:40:00': [1412.1, 1409.0],
#   '2020-08-06 15:18:00': [1406.75, 1405.95],
#   '2020-08-07 11:19:00': [1401.3, 1398.5],
#   '2020-08-11 15:11:00': [1413.55, 1412.15],
#   '2020-08-12 11:59:00': [1397.1, 1396.15],
#   '2020-08-13 10:35:00': [1418.45, 1417.9],
#   '2020-08-13 14:14:00': [1414.0, 1410.15],
#   '2020-08-14 13:53:00': [1391.2, 1386.0],
#   '2020-08-17 13:29:00': [1380.8, 1380.4],
#   '2020-08-18 13:07:00': [1422.2, 1421.55],
#   '2020-08-19 13:20:00': [1435.0, 1434.95],
#   '2020-08-19 14:36:00': [1430.0, 1428.0],
#   '2020-08-21 11:42:00': [1423.0, 1422.1],
#   '2020-08-21 12:47:00': [1425.3, 1423.2],
#   '2020-08-21 15:00:00': [1424.1, 1422.9],
#   '2020-08-24 10:37:00': [1428.05, 1427.9],
#   '2020-08-24 10:56:00': [1424.15, 1423.8],
#   '2020-08-24 15:11:00': [1422.5, 1421.95],
#   '2020-08-25 11:20:00': [1412.2, 1411.4],
#   '2020-08-26 10:03:00': [1398.55, 1394.9],
#   '2020-08-27 12:02:00': [1398.3, 1397.4],
#   '2020-08-27 15:16:00': [1399.85, 1398.95],
#   '2020-08-28 13:02:00': [1402.0, 1401.0],
#   '2020-08-28 13:21:00': [1398.8, 1398.65],
#   '2020-08-31 11:38:00': [1372.0, 1369.1],
#   '2020-08-31 13:02:00': [1353.6, 1352.75],
#   '2020-08-31 14:15:00': [1328.7, 1327.7],
#   '2020-09-03 10:17:00': [1344.25, 1343.4],
#   '2020-09-03 11:30:00': [1340.0, 1338.95],
#   '2020-09-04 14:12:00': [1315.45, 1315.3],
#   '2020-09-14 13:00:00': [1371.6, 1370.0],
#   '2020-09-16 11:11:00': [1388.1, 1386.55],
#   '2020-09-21 15:04:00': [1379.7, 1378.9],
#   '2020-09-22 13:06:00': [1378.7, 1378.25],
#   '2020-09-22 13:29:00': [1376.1, 1374.35],
#   '2020-09-23 13:26:00': [1336.0, 1335.2],
#   '2020-09-23 13:55:00': [1331.35, 1331.0],
#   '2020-09-24 10:27:00': [1328.7, 1325.95],
#   '2020-09-24 11:38:00': [1325.1, 1324.45],
#   '2020-09-29 10:07:00': [1378.8, 1378.0],
#   '2020-09-29 12:37:00': [1368.65, 1367.0],
#   '2020-10-01 10:09:00': [1400.2, 1399.0],
#   '2020-10-05 09:53:00': [1411.1, 1409.5],
#   '2020-10-07 14:51:00': [1486.1, 1484.85],
#   '2020-10-09 13:05:00': [1512.95, 1511.4],
#   '2020-10-15 14:11:00': [1521.8, 1521.35],
#   '2020-10-20 12:37:00': [1575.85, 1575.1],
#   '2020-10-26 12:41:00': [1591.4, 1590.75],
#   '2020-11-02 12:15:00': [1655.05, 1655.0],
#   '2020-11-06 10:52:00': [1688.25, 1686.75],
#   '2020-11-06 12:15:00': [1676.55, 1675.95],
#   '2020-11-06 13:02:00': [1670.0, 1668.7],
#   '2020-11-06 13:14:00': [1659.0, 1657.2],
#   '2020-11-06 14:27:00': [1660.3, 1660.2],
#   '2020-11-09 13:18:00': [1656.0, 1655.2],
#   '2020-11-10 12:17:00': [1682.5, 1681.9],
#   '2020-11-17 11:59:00': [1712.8, 1712.0],
#   '2020-11-17 12:14:00': [1705.05, 1702.35],
#   '2020-11-17 14:59:00': [1700.25, 1699.5],
#   '2020-11-18 09:57:00': [1691.0, 1689.95],
#   '2020-11-18 12:40:00': [1697.3, 1695.0],
#   '2020-11-18 13:56:00': [1698.1, 1696.05]}

In [10]:
plot(check_df)